In [1]:
from google.colab import auth
auth.authenticate_user()
print("Authenticated")

Authenticated


In [2]:
from google.cloud import bigquery
client = bigquery.Client(project='bq-rideshare')

In [3]:
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

In [4]:
tables = list(client.list_tables(dataset))

for table in tables:
  print(table.table_id)

taxi_trips


In [6]:
table_ref = dataset_ref.table('taxi_trips')
table = client.get_table(table_ref)
# # alternate method of above
# table = client.get_table(dataset.table('taxi_trips'))

In [16]:
# metadata for table
[command for command in dir(table) if not command.startswith('_')]

['clustering_fields',
 'created',
 'dataset_id',
 'description',
 'encryption_configuration',
 'etag',
 'expires',
 'external_data_configuration',
 'friendly_name',
 'from_api_repr',
 'from_string',
 'full_table_id',
 'labels',
 'location',
 'modified',
 'num_bytes',
 'num_rows',
 'partition_expiration',
 'partitioning_type',
 'path',
 'project',
 'reference',
 'require_partition_filter',
 'schema',
 'self_link',
 'streaming_buffer',
 'table_id',
 'table_type',
 'time_partitioning',
 'to_api_repr',
 'to_bqstorage',
 'view_query',
 'view_use_legacy_sql']

In [7]:
table.schema

[SchemaField('unique_key', 'STRING', 'REQUIRED', 'Unique identifier for the trip.', ()),
 SchemaField('taxi_id', 'STRING', 'REQUIRED', 'A unique identifier for the taxi.', ()),
 SchemaField('trip_start_timestamp', 'TIMESTAMP', 'NULLABLE', 'When the trip started, rounded to the nearest 15 minutes.', ()),
 SchemaField('trip_end_timestamp', 'TIMESTAMP', 'NULLABLE', 'When the trip ended, rounded to the nearest 15 minutes.', ()),
 SchemaField('trip_seconds', 'INTEGER', 'NULLABLE', 'Time of the trip in seconds.', ()),
 SchemaField('trip_miles', 'FLOAT', 'NULLABLE', 'Distance of the trip in miles.', ()),
 SchemaField('pickup_census_tract', 'INTEGER', 'NULLABLE', 'The Census Tract where the trip began. For privacy, this Census Tract is not shown for some trips.', ()),
 SchemaField('dropoff_census_tract', 'INTEGER', 'NULLABLE', 'The Census Tract where the trip ended. For privacy, this Census Tract is not shown for some trips.', ()),
 SchemaField('pickup_community_area', 'INTEGER', 'NULLABLE', '

In [15]:
df = client.list_rows(table, max_results=5).to_dataframe()
df

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,7b3e74418111fef19fd73835853d9c1cebd727d5,fa62820eb95bd0b9cc1a0aaf6cf26730aee22ec578db91...,2014-04-01 16:00:00+00:00,2014-04-01 16:00:00+00:00,0,0.0,None,None,None,None,...,0.0,6.45,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None
1,9d2eefe122d32a385207087a71dd33deed0eea96,89612cda57630834fb854b4e42206d3478145557d975ce...,2014-03-06 21:30:00+00:00,2014-03-06 21:30:00+00:00,0,0.0,None,None,None,None,...,0.0,12.05,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None
2,a8410244d022d258ae357f6ede25d0162efd675f,36dee7e88e2048dfeb8a98c9e420a1600544a8d4f21fe6...,2014-04-18 10:45:00+00:00,2014-04-18 10:45:00+00:00,0,0.0,None,None,None,None,...,0.0,12.55,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None
3,c590e1ea0a0998541e049de913e4a26499955123,85d86e614d844575634820c6e19498d1cfd59bdecc396b...,2015-02-05 15:30:00+00:00,2015-02-05 15:45:00+00:00,1020,2.5,None,None,None,None,...,0.0,9.85,Cash,None,None,None,None,None,None,None
4,bdd767ee32e7038e4c1bcac28cd516cc7cb549b8,f4bc2acfbb68d505b7ca37f8553ab1fc28a2686429b55d...,2014-03-28 09:45:00+00:00,2014-03-28 09:45:00+00:00,0,0.0,None,None,None,None,...,0.0,16.45,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,None,None,None,None,None,None


In [14]:
type(df)

pandas.core.frame.DataFrame

In [18]:
df.columns

Index(['unique_key', 'taxi_id', 'trip_start_timestamp', 'trip_end_timestamp',
       'trip_seconds', 'trip_miles', 'pickup_census_tract',
       'dropoff_census_tract', 'pickup_community_area',
       'dropoff_community_area', 'fare', 'tips', 'tolls', 'extras',
       'trip_total', 'payment_type', 'company', 'pickup_latitude',
       'pickup_longitude', 'pickup_location', 'dropoff_latitude',
       'dropoff_longitude', 'dropoff_location'],
      dtype='object')

In [19]:
sel_cols = ('unique_key', 'taxi_id', 'trip_start_timestamp',
       'trip_seconds', 'trip_miles', 'pickup_census_tract',
       'dropoff_census_tract', 'pickup_community_area',
       'dropoff_community_area', 'fare', 'tips', 'tolls', 'extras',
       'trip_total', 'payment_type', 'company')

schema_subset = [col for col in table.schema if col.name in sel_cols]
results = [x for x in client.list_rows(table, start_index=100, selected_fields=schema_subset, max_results=10)]

In [20]:
print(results)
for i in results:
    print(dict(i))

[Row(('28aa3a5257de9e79fb02662b040c1fe282771c05', '950135778d48278175cc7c82f3aedb890b0c4f2bb06b113f494f5ab27b97a5b5ad13c7afacba6b3b37e70076e81ef6f8bf407270159a80368c64f9c784b95a5c', datetime.datetime(2014, 4, 1, 17, 0, tzinfo=<UTC>), 0, 0.0, None, None, None, None, 36.25, 9.06, 0.0, 0.0, 45.31, 'Credit Card', 'Chicago Elite Cab Corp. (Chicago Carriag'), {'unique_key': 0, 'taxi_id': 1, 'trip_start_timestamp': 2, 'trip_seconds': 3, 'trip_miles': 4, 'pickup_census_tract': 5, 'dropoff_census_tract': 6, 'pickup_community_area': 7, 'dropoff_community_area': 8, 'fare': 9, 'tips': 10, 'tolls': 11, 'extras': 12, 'trip_total': 13, 'payment_type': 14, 'company': 15}), Row(('ae3be97a0bef8b820eadf89b8448fb0a4177b7f5', 'bb2b1096811ecd58e2e215aad17c5f1a7083521937c0ce15c8f6893d549b99bd2476a1bbdbf9bf736215816d27c756a2233213bbd1f976daba2ae1debd2b42e9', datetime.datetime(2014, 3, 9, 14, 45, tzinfo=<UTC>), 0, 0.0, None, None, None, None, 37.05, 7.41, 0.0, 0.0, 44.46, 'Credit Card', 'Chicago Elite Cab Corp

In [21]:
# find total GB
BYTES_PER_GB = 2**30
table.num_bytes / BYTES_PER_GB

73.11011400539428

In [32]:
table.num_rows

201741514

In [31]:
sql_query = ('''SELECT *
                FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                WHERE trip_start_timestamp > '2019-01-01 0:00:00'
                AND trip_start_timestamp < '2019-12-31 11:59:59'
                ''')
taxi_records_2019 = client.query(sql_query).to_dataframe()

KeyboardInterrupt: ignored

In [30]:
taxi_records_2019.num_bytes/ BYTES_PER_GB

NameError: ignored

# New Section